In [1]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from agents.tec_an import *
from agents.data_agent import *
from agents.stock_agent import *
import numpy as np
from data_util import *
import pandas as pd
from backtest import *
from joblib import dump, load
from datetime import datetime, timedelta
from bitstamp import *
from process_live_trade import *
from model_winner_select import *
from eval_model import *
from cache_providers import *

base_dir = "data/"

In [2]:
currency = "ethusd"

full_data = base_dir + "backtest/"
full_data = base_dir + "4jul21/"
full_data = base_dir + "ethusd/"
#full_data = base_dir + "btcusd/"
data_gen = DataGenerator(random = False, base_dir = full_data)
data_gen.rewind()
data_count = (data_gen.steps - 100)
#data_count = 200000
raw = data_gen.next()

hot_load = True

In [3]:
path = "model/"
files = os.listdir(path)
models = []
for file in files:
    models.append(f"{path}{file}")

models = ['model/ber_30_20']
    
val_start = 1629507199#int(raw['timestamp'])

In [4]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online_path = f'data/online{file_prefix}_{minutes}'
        
    return load(online_path)  

def get_provider(minu, win):
    tec = TecAn(windows = win, windows_limit = 100)
    sourceDataGenerator = SourceDataGenerator(tec = tec)
    provider = get_online_data(
            minutes = minu, 
            source_data_generator=sourceDataGenerator, 
            load_from_disk = True, 
            file_prefix=win
            )
        
    return provider

In [5]:
trained = load("best_models/winner_3_ethusd")

In [6]:
print_evalueted_model(trained)

ethusd -> 107.31243683689513 | 115 - 34 | stop_loss: -0.5
Eveluated timeframe: from 2021-09-08 12:44:47(1631105087) to 2021-09-11 12:44:47(1631364287)
windows = 40 minutes = 3 steps_ahead = 3
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFE(estimator=ExtraTreesClassifier(max_features=0.3,
                                                                     random_state=42),
                                      step=0.7500000000000001)),
                                 ('m',
                                  RandomForestClassifier(bootstrap=False,
                                                         criterion='entropy',
                                                         max_features=0.1,
                                                         random_state=42))]))])


In [7]:
trained_model = trained

In [8]:
minutes = trained_model.model_detail.data_detail.minutes
windows = trained_model.model_detail.data_detail.windows
steps_ahead = trained_model.model_detail.data_detail.steps_ahead
provider = get_provider(minu = minutes, win = windows)  
stop_loss = -1 #trained_model.stop_loss

model = trained_model.model_detail.model


In [9]:
import numpy as np
import matplotlib.pyplot as plt
def get_charts(stock_data):
    fig = plt.figure()
    ax = plt.axes()
    x = stock_data
    plt.plot(x, linewidth=1)
    #plt.grid(b=True, color='aqua', alpha=0.3, linestyle='-.', linewidth=2)
    plt.show()


In [10]:

on_raw_data = lambda raw: print(raw)

data_gen.rewind()

first = data_gen.next()
last = data_gen.get_from_index(data_gen.steps)

In [11]:
val_start = int(first['timestamp'])
val_end = int(last['timestamp'])

start_string = f"{pd.to_datetime(val_start, unit='s')}({val_start})"
end_string = f"{pd.to_datetime(val_end, unit='s')}({val_end})"
print(f"From {start_string} to {end_string}")

From 2021-03-19 23:24:01(1616196241) to 2021-03-21 14:30:26(1616337026)


In [12]:
cache = CacheProvider([currency], verbose = True)

online = cache.get_provider(minutes, windows, val_start = val_start, val_end = val_end)


2021-09-15 21:21:32.979489 CacheProvider: Added: 3-40


In [20]:

agent_raw, back_raw, stock_raw = get_agent(minutes = minutes,
                                win = windows,
                                step = steps_ahead,
                                currency = currency,
                                hot_load = False,
                                model = model,
                                verbose = True,
                                timestamp = val_start,
                                stop_loss = stop_loss,
                                simulate_on_price = True)
prices = []

In [21]:
agent_raw.save_history = True
agent_raw.verbose = True
stock_raw.verbose = True

In [22]:
data_gen.rewind()
#agent_raw.save_history = True
#stock_raw.verbose = True

#data_count
for i in range(data_count):
    raw = data_gen.next()
    price = raw[PRICE_KEY]
    amount = raw[AMOUNT_KEY]
    timestamp = int(raw[TIMESTAMP_KEY])
    microtimestamp = raw[MICROTIMESTAMP_KEY]
    if (timestamp >= 1616205445):
        print(f"{pd.to_datetime(timestamp, unit='s')}({timestamp}) - {price}/{amount}")
    agent_raw.on_new_raw_data(raw)

2021-09-15 21:23:25.791287 DataAgent: last_index: -1 current_index: 2021-03-19 23:24:00  price: 1809.19 - last_price: 1809.19 = price: 1809.19
2021-09-15 21:23:25.830040 ModelAgent(1809.19): 2021-03-19 23:24:01(1616196241) DOWN
2021-09-15 21:23:25.938274 DataAgent: last_index: 2021-03-19 23:24:00 current_index: 2021-03-19 23:27:00  price: 1808.71 - last_price: 1808.71 = price: 1808.71
2021-09-15 21:23:25.972286 ModelAgent(1808.71): 2021-03-19 23:27:55(1616196475) UP
2021-09-15 21:23:25.972445 BackTest: Bought: 1808.71
2021-09-15 21:23:26.059494 DataAgent: last_index: 2021-03-19 23:27:00 current_index: 2021-03-19 23:30:00  price: 1810.0 - last_price: 1810.0 = price: 1810.0
2021-09-15 21:23:26.094193 ModelAgent(1810.0): 2021-03-19 23:30:45(1616196645) DOWN
2021-09-15 21:23:26.094354 BackTest: SOLD >>>> Result: PROFIT 0.0713% total: 100.0713215496127
2021-09-15 21:23:26.179867 DataAgent: last_index: 2021-03-19 23:30:00 current_index: 2021-03-19 23:33:00  price: 1811.6 - last_price: 1811.6

2021-09-15 21:23:31.226764 DataAgent: last_index: 2021-03-20 01:06:00 current_index: 2021-03-20 01:09:00  price: 1819.08 - last_price: 1819.08 = price: 1819.08
2021-09-15 21:23:31.264632 ModelAgent(1819.08): 2021-03-20 01:10:25(1616202625) DOWN
2021-09-15 21:23:31.348776 DataAgent: last_index: 2021-03-20 01:09:00 current_index: 2021-03-20 01:12:00  price: 1818.27 - last_price: 1818.27 = price: 1818.27
2021-09-15 21:23:31.384629 ModelAgent(1818.27): 2021-03-20 01:13:41(1616202821) DOWN
2021-09-15 21:23:31.427083 DataAgent: last_index: 2021-03-20 01:12:00 current_index: 2021-03-20 01:15:00  price: 1818.5 - last_price: 1818.5 = price: 1818.5
2021-09-15 21:23:31.469429 ModelAgent(1818.5): 2021-03-20 01:15:11(1616202911) DOWN
2021-09-15 21:23:31.595892 DataAgent: last_index: 2021-03-20 01:15:00 current_index: 2021-03-20 01:18:00  price: 1815.66 - last_price: 1815.66 = price: 1815.66
2021-09-15 21:23:31.629764 ModelAgent(1815.66): 2021-03-20 01:19:21(1616203161) DOWN
2021-09-15 21:23:31.7300

2021-03-20 01:58:22(1616205502) - 1824.0/0.04072541
2021-03-20 01:58:23(1616205503) - 1824.0/0.04072541
2021-03-20 01:58:23(1616205503) - 1824.0/0.04072541
2021-03-20 01:58:23(1616205503) - 1824.0/0.04072541
2021-03-20 01:58:23(1616205503) - 1824.0/0.04072541
2021-03-20 01:58:23(1616205503) - 1824.0/0.04072541
2021-03-20 01:58:24(1616205504) - 1824.0/0.04072541
2021-03-20 01:58:24(1616205504) - 1824.0/0.04072541
2021-03-20 01:58:24(1616205504) - 1824.0/0.04072541
2021-03-20 01:58:24(1616205504) - 1824.0/0.04072541
2021-03-20 01:58:24(1616205504) - 1824.0/0.04072541
2021-03-20 01:58:25(1616205505) - 1824.0/0.04072541
2021-03-20 01:58:25(1616205505) - 1824.0/0.04072541
2021-03-20 01:58:25(1616205505) - 1824.0/0.04072541
2021-03-20 01:58:25(1616205505) - 1824.0/0.04072541
2021-03-20 01:58:26(1616205506) - 1824.0/0.04072541
2021-03-20 01:58:27(1616205507) - 1824.0/0.04072541
2021-03-20 01:58:28(1616205508) - 1824.0/0.04072541
2021-03-20 01:58:28(1616205508) - 1824.0/0.04072541
2021-03-20 0

2021-03-20 01:59:34(1616205574) - 1824.38/0.18463725
2021-03-20 01:59:34(1616205574) - 1824.38/0.18463725
2021-03-20 01:59:34(1616205574) - 1824.38/0.18463725
2021-03-20 01:59:34(1616205574) - 1824.38/0.18463725
2021-03-20 01:59:34(1616205574) - 1824.38/0.18463725
2021-03-20 01:59:35(1616205575) - 1824.38/0.18463725
2021-03-20 01:59:35(1616205575) - 1824.38/0.18463725
2021-03-20 01:59:35(1616205575) - 1824.38/0.18463725
2021-03-20 01:59:35(1616205575) - 1824.38/0.18463725
2021-03-20 01:59:35(1616205575) - 1824.38/0.18463725
2021-03-20 01:59:36(1616205576) - 1824.38/0.18463725
2021-03-20 01:59:36(1616205576) - 1824.38/0.18463725
2021-03-20 01:59:36(1616205576) - 1824.38/0.18463725
2021-03-20 01:59:37(1616205577) - 1824.38/0.18463725
2021-03-20 01:59:37(1616205577) - 1824.38/0.18463725
2021-03-20 01:59:37(1616205577) - 1824.38/0.18463725
2021-03-20 01:59:37(1616205577) - 1824.38/0.18463725
2021-03-20 01:59:37(1616205577) - 1824.38/0.18463725
2021-03-20 01:59:38(1616205578) - 1824.38/0.18

2021-03-20 02:00:37(1616205637) - 1823.1/0.0001225
2021-03-20 02:00:38(1616205638) - 1823.1/0.0001225
2021-03-20 02:00:38(1616205638) - 1823.1/0.0001225
2021-03-20 02:00:38(1616205638) - 1823.1/0.0001225
2021-03-20 02:00:38(1616205638) - 1823.1/0.0001225
2021-03-20 02:00:38(1616205638) - 1823.1/0.0001225
2021-03-20 02:00:39(1616205639) - 1823.1/0.0001225
2021-03-20 02:00:39(1616205639) - 1823.1/0.0001225
2021-03-20 02:00:39(1616205639) - 1823.1/0.0001225
2021-03-20 02:00:39(1616205639) - 1823.1/0.0001225
2021-03-20 02:00:40(1616205640) - 1823.1/0.0001225
2021-03-20 02:00:40(1616205640) - 1823.1/0.0001225
2021-03-20 02:00:40(1616205640) - 1823.1/0.0001225
2021-03-20 02:00:40(1616205640) - 1823.1/0.0001225
2021-03-20 02:00:41(1616205641) - 1823.1/0.0001225
2021-03-20 02:00:41(1616205641) - 1823.1/0.0001225
2021-03-20 02:00:41(1616205641) - 1823.1/0.0001225
2021-03-20 02:00:41(1616205641) - 1823.1/0.0001225
2021-03-20 02:00:42(1616205642) - 1823.1/0.0001225
2021-03-20 02:00:42(1616205642)

...

2021-03-20 02:04:16(1616205856) - 1823.61/8.8
2021-03-20 02:04:16(1616205856) - 1823.61/8.8
2021-03-20 02:04:16(1616205856) - 1823.61/8.8
2021-03-20 02:04:17(1616205857) - 1823.61/8.8
2021-03-20 02:04:17(1616205857) - 1823.61/8.8
2021-03-20 02:04:18(1616205858) - 1823.61/8.8
2021-03-20 02:04:18(1616205858) - 1823.61/8.8
2021-03-20 02:04:18(1616205858) - 1823.61/8.8
2021-03-20 02:04:18(1616205858) - 1823.61/8.8
2021-03-20 02:04:18(1616205858) - 1823.61/8.8
2021-03-20 02:04:19(1616205859) - 1823.61/8.8
2021-03-20 02:04:19(1616205859) - 1823.61/8.8
2021-03-20 02:04:19(1616205859) - 1823.61/8.8
2021-03-20 02:04:19(1616205859) - 1823.61/8.8
2021-03-20 02:04:19(1616205859) - 1823.61/8.8
2021-03-20 02:04:20(1616205860) - 1823.61/8.8
2021-03-20 02:04:20(1616205860) - 1823.61/8.8
2021-03-20 02:04:20(1616205860) - 1823.61/8.8
2021-03-20 02:04:20(1616205860) - 1823.61/8.8
2021-03-20 02:04:20(1616205860) - 1823.61/8.8
2021-03-20 02:04:21(1616205861) - 1823.61/8.8
2021-03-20 02:04:21(1616205861) - 

2021-03-20 02:05:27(1616205927) - 1823.61/8.8
2021-03-20 02:05:28(1616205928) - 1823.61/8.8
2021-03-20 02:05:28(1616205928) - 1823.61/8.8
2021-03-20 02:05:28(1616205928) - 1823.61/8.8
2021-03-20 02:05:28(1616205928) - 1823.61/8.8
2021-03-20 02:05:29(1616205929) - 1823.61/8.8
2021-03-20 02:05:29(1616205929) - 1823.61/8.8
2021-03-20 02:05:29(1616205929) - 1823.61/8.8
2021-03-20 02:05:30(1616205930) - 1823.61/8.8
2021-03-20 02:05:30(1616205930) - 1823.61/8.8
2021-03-20 02:05:30(1616205930) - 1823.61/8.8
2021-03-20 02:05:30(1616205930) - 1823.61/8.8
2021-03-20 02:05:31(1616205931) - 1823.61/8.8
2021-03-20 02:05:31(1616205931) - 1823.61/8.8
2021-03-20 02:05:31(1616205931) - 1823.61/8.8
2021-03-20 02:05:31(1616205931) - 1823.61/8.8
2021-03-20 02:05:31(1616205931) - 1823.61/8.8
2021-03-20 02:05:32(1616205932) - 1823.61/8.8
2021-03-20 02:05:32(1616205932) - 1823.61/8.8
2021-03-20 02:05:32(1616205932) - 1823.61/8.8
2021-03-20 02:05:32(1616205932) - 1823.61/8.8
2021-03-20 02:05:32(1616205932) - 

2021-03-20 02:06:42(1616206002) - 1823.61/8.8
2021-03-20 02:06:42(1616206002) - 1823.61/8.8
2021-03-20 02:06:43(1616206003) - 1823.61/8.8
2021-03-20 02:06:43(1616206003) - 1823.61/8.8
2021-03-20 02:06:44(1616206004) - 1823.61/8.8
2021-03-20 02:06:44(1616206004) - 1823.61/8.8
2021-03-20 02:06:44(1616206004) - 1823.61/8.8
2021-03-20 02:06:45(1616206005) - 1823.61/8.8
2021-03-20 02:06:45(1616206005) - 1823.61/8.8
2021-03-20 02:06:45(1616206005) - 1823.61/8.8
2021-03-20 02:06:45(1616206005) - 1823.61/8.8
2021-03-20 02:06:45(1616206005) - 1823.61/8.8
2021-03-20 02:06:46(1616206006) - 1823.61/8.8
2021-03-20 02:06:46(1616206006) - 1823.61/8.8
2021-03-20 02:06:46(1616206006) - 1823.61/8.8
2021-03-20 02:06:46(1616206006) - 1823.61/8.8
2021-03-20 02:06:46(1616206006) - 1823.61/8.8
2021-03-20 02:06:47(1616206007) - 1823.61/8.8
2021-03-20 02:06:47(1616206007) - 1823.61/8.8
2021-03-20 02:06:47(1616206007) - 1823.61/8.8
2021-03-20 02:06:47(1616206007) - 1823.61/8.8
2021-03-20 02:06:47(1616206007) - 

SystemExit: AGENT Diff 360 Timeframe: 180 last_index: 2021-03-20 02:00:00(1616205600) current_index: 2021-03-20 02:06:00(1616205960)

/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
agent_raw.report()

In [ ]:
val_start = int(first['timestamp'])
val_end = int(raw['timestamp'])

start_string = pd.to_datetime(val_start, unit='s')
end_string = pd.to_datetime(val_end, unit='s')

print(f"{start_string}({val_start}) -> {end_string}({val_end})")

In [ ]:
back_raw.on_down(back_raw.buy_price, back_raw.buy_price)
back_raw.report()

In [ ]:
back_raw.on_down(back_raw.buy_price, back_raw.buy_price)
back_raw.report()

In [ ]:
agent_raw.ohlc[-50:]["open"].plot()
#agent.ohlc[1:]

In [ ]:
x_list, price_list, time_list = online.load_val_data(currency)

In [ ]:
get_charts(price_list)
print(len(price_list))

In [ ]:
agent_array, back_array, stock_array = get_agent(minutes = minutes,
                                win = windows,
                                step = step,
                                currency = currency,
                                hot_load = hot_load,
                                model = model,
                                timestamp = int(first['timestamp']),
                                simulate_on_price = True,
                                verbose = False)

In [ ]:
agent_array.save_history = True
stock_array.verbose = True
for idx in range(len(price_list)):
    price = price_list[idx]
    time = time_list[idx]
    order = [[f"{price}", f"{price}"]]
    amount = 0.0
    agent_array.process_data(price, amount, time, order, order)